In [1]:
import pandas as pd
import sys
sys.path.append('../../ResponsibleAIToolbox-Mitigation/')
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek

from errorsmitigation.dataprocessing import DataRebalance
from databalanceanalysis.databalanceanalysis.utils import undummify

In [2]:
data_dir = '../datasets/hr_promotion'
df =  pd.read_csv(data_dir + '/train.csv')
# data_dir = '../datasets/'
# df = pd.read_csv(data_dir + 'AdultCensusIncome.csv') 
# df['income'] = df['income'].apply(lambda x: 0 if x == "<=50K"  else 1)
cols_of_interest = ['education', 'gender']
# cols_of_interest = ['sex', 'race']
label_col = 'is_promoted'
# label_col = 'income'
seed = 42
df.shape
df = df.dropna()
df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [3]:
from databalanceanalysis.databalanceanalysis.feature_measures import FeatureBalanceMeasure

feature_measures = FeatureBalanceMeasure( cols_of_interest, label_col)

feat_measures = feature_measures.measures(df)
feat_measures

,classA,classB,feature_name,Measures.DEMOGRAPHIC_PARITY,Measures.POINTWISE_MUTUAL_INFO,Measures.SD_COEF,Measures.JACCARD_INDEX,Measures.KR_CORRELATION,Measures.LOG_LIKELIHOOD,Measures.TTEST,Measures.TTEST_PVALUE
0,Master's & above,Bachelor's,education,0.017048,0.189273,0.004262,0.004978,4.963227,-0.624911,0.260550,0.409406
1,Master's & above,Below Secondary,education,0.020262,0.229347,0.069203,0.075546,-27.230132,3.704725,-2.639010,0.059292
2,Bachelor's,Below Secondary,education,0.003214,0.040074,0.064941,0.070567,-32.193358,4.329636,-2.899560,0.050603
0,f,m,gender,0.007294,0.082555,-0.003747,-0.004365,5.034651,-0.744275,0.264311,0.417748


In [12]:
from databalanceanalysis.databalanceanalysis.distribution_measures import DistributionBalanceMeasure

dist_measures = DistributionBalanceMeasure( cols_of_interest)
dist_measures.measures(df)

f_obs
          education  count
0        Bachelor's  33404
1   Below Secondary    458
2  Master's & above  14798
f_obs
  gender  count
0      f  14808
1      m  33852


,feature_name,Measures.KL_DIVERGENCE,Measures.JS_DISTANCE,Measures.WS_DISTANCE,Measures.INF_NORM_DISTANCE,Measures.TOTAL_VARIANCE_DISTANCE,Measures.CHISQ_PVALUE,Measures.CHISQ
0,education,0.434455,0.358855,0.235430,0.353144,0.353144,0.0,33646.891739
1,gender,0.078670,0.141651,0.195684,0.195684,0.195684,0.0,7453.225154


In [13]:
from databalanceanalysis.databalanceanalysis.aggregate_measures import AggregateBalanceMeasure

agg_measures = AggregateBalanceMeasure( cols_of_interest)
agg_measures.measures(df)

,Measures.THEIL_L_INDEX,Measures.THEIL_T_INDEX,Measures.ATKINSON_INDEX
0,1.046355,0.513502,0.648784


In [4]:
smote_tomek = SMOTETomek()
smote = SMOTE()
tomek = TomekLinks()
dummy_df = pd.get_dummies(df, prefix_sep = "-")

In [16]:
# undummify(dummy_df, prefix_sep = "-", col_list = ['race'])
# undummify(dummy_df, prefix_sep="-", col_list = ['sex'])
gender_df = undummify(dummy_df, prefix_sep = "-", col_list = ['gender'])
gender_df

,gender,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,...,region-region_6,region-region_7,region-region_8,region-region_9,education-Bachelor's,education-Below Secondary,education-Master's & above,recruitment_channel-other,recruitment_channel-referred,recruitment_channel-sourcing
0,f,65438,1,35,5.000000,8,1,0,49,0,...,0,1,0,0,0,0,1,0,0,1
1,m,65141,1,30,5.000000,4,0,0,60,0,...,0,0,0,0,1,0,0,1,0,0
2,m,7513,1,34,3.000000,7,0,0,50,0,...,0,0,0,0,1,0,0,0,0,1
3,m,2542,2,39,1.000000,10,0,0,50,0,...,0,0,0,0,1,0,0,1,0,0
4,m,48945,1,45,3.000000,2,0,0,73,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67699,f,33797,1,33,5.000000,7,1,0,55,0,...,0,0,0,0,0,0,1,0,0,0
67700,f,22288,1,33,4.111154,8,0,0,74,0,...,0,0,0,0,0,0,1,0,0,0
67701,f,58455,1,32,3.000000,7,0,0,67,0,...,0,0,0,0,1,0,0,0,0,1
67702,f,26205,1,39,3.393670,8,1,0,53,0,...,0,0,0,0,0,0,0,1,0,0


In [31]:
gender_df = undummify(dummy_df, prefix_sep = "-", col_list = ['gender'])
#gender_df
data_balance_smote =  DataRebalance(gender_df, 'gender', 'auto', 42, None, smote, None)
smote_df = data_balance_smote.Rebalance()
print(smote_df.shape)
# smote_df
dummy_df = pd.get_dummies(smote_df, prefix_sep ="-")
education_df = undummify(dummy_df, prefix_sep = "-", col_list = ['education'])
print(education_df.shape)
data_balance_smote_2 =  DataRebalance(education_df, 'education', None, None, None, smote, None)
smote_df_2 = data_balance_smote_2.Rebalance()
print(smote_df_2.shape)

(67704, 59)
(67704, 58)
(151230, 58)


In [26]:
education_df['education'].value_counts()

Bachelor's          50410
Master's & above    16832
Below Secondary       462
Name: education, dtype: int64

In [30]:
education_df.tail()

,education,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,...,region-region_5,region-region_6,region-region_7,region-region_8,region-region_9,recruitment_channel-other,recruitment_channel-referred,recruitment_channel-sourcing,gender-f,gender-m
67699,Master's & above,33797,1,33,5.000000,7,1,0,55,0,...,0,0,0,0,0,0,0,0,1,0
67700,Master's & above,22288,1,33,4.111154,8,0,0,74,0,...,0,0,0,0,0,0,0,0,1,0
67701,Bachelor's,58455,1,32,3.000000,7,0,0,67,0,...,0,0,0,0,0,0,0,1,1,0
67702,Bachelor's,26205,1,39,3.393670,8,1,0,53,0,...,0,0,0,0,0,1,0,0,1,0
67703,Master's & above,30218,1,45,2.845543,13,0,0,69,0,...,0,0,0,0,0,0,0,0,1,0


In [27]:
smote_df_2['education'].value_counts()

Master's & above    50410
Bachelor's          50410
Below Secondary     50410
Name: education, dtype: int64

In [29]:
smote_df_2.head()

,education,employee_id,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,...,region-region_5,region-region_6,region-region_7,region-region_8,region-region_9,recruitment_channel-other,recruitment_channel-referred,recruitment_channel-sourcing,gender-f,gender-m
0,Master's & above,65438,1,35,5.0,8,1,0,49,0,...,0,0,1,0,0,0,0,1,1,0
1,Bachelor's,65141,1,30,5.0,4,0,0,60,0,...,0,0,0,0,0,1,0,0,0,1
2,Bachelor's,7513,1,34,3.0,7,0,0,50,0,...,0,0,0,0,0,0,0,1,0,1
3,Bachelor's,2542,2,39,1.0,10,0,0,50,0,...,0,0,0,0,0,1,0,0,0,1
4,Bachelor's,48945,1,45,3.0,2,0,0,73,0,...,0,0,0,0,0,1,0,0,0,1


In [9]:
new_df = undummify(smote_df_2, prefix_sep="-")

In [11]:
feature_measures.measures(new_df)

,classA,classB,feature_name,Measures.DEMOGRAPHIC_PARITY,Measures.POINTWISE_MUTUAL_INFO,Measures.SD_COEF,Measures.JACCARD_INDEX,Measures.KR_CORRELATION,Measures.LOG_LIKELIHOOD,Measures.TTEST,Measures.TTEST_PVALUE
0,Master's & above,Bachelor's,education,-0.021107,-0.457321,-0.019168,-0.020916,-0.165337,-0.457321,0.000000,0.500000
1,Master's & above,Below Secondary,education,0.029161,1.614902,0.026482,0.027569,0.228426,1.614902,0.000000,0.500000
2,Bachelor's,Below Secondary,education,0.050268,2.072224,0.045650,0.048485,0.393763,2.072224,0.000000,0.500000
0,f,m,gender,-0.027347,-0.813668,-0.025057,-0.026805,-2.930860,-0.559770,-0.049549,0.484241


In [14]:
dist_measures.measures(new_df)

f_obs
          education  count
0        Bachelor's  50410
1   Below Secondary  50410
2  Master's & above  50410
f_obs
  gender  count
0      f  85163
1      m  66067


,feature_name,Measures.KL_DIVERGENCE,Measures.JS_DISTANCE,Measures.WS_DISTANCE,Measures.INF_NORM_DISTANCE,Measures.TOTAL_VARIANCE_DISTANCE,Measures.CHISQ_PVALUE,Measures.CHISQ
0,education,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000
1,gender,0.007994,0.044748,0.063136,0.063136,0.063136,0.0,2411.275646


In [15]:
agg_measures.measures(new_df)

,Measures.THEIL_L_INDEX,Measures.THEIL_T_INDEX,Measures.ATKINSON_INDEX
0,0.009425,0.009334,0.00938


## IGNORE BELOW FOR NOW

In [8]:
race_df = undummify(dummy_df, prefix_sep = "-", col_list = ['race'])
data_rebalance_smote =  DataRebalance(race_df, 'race', None, None, None, smote, None)
data_rebalance_tomek = DataRebalance(race_df, 'race', None, None, None, None, None)
data_rebalance_smote_tomek = DataRebalance(race_df, 'race', None, None, None, None, None)
smote_df = data_rebalance_smote.Rebalance()
tomek_df = data_rebalance_tomek.Rebalance()
smote_tomek_df = data_rebalance_smote_tomek.Rebalance()


ValueError: attempt to get argmax of an empty sequence

In [ ]:
smote_df

In [ ]:
# df = undummify(df, "-")
# smote_df = undummify(smote_df, "-")
# smote_tomek_df = undummify(smote_tomek_df, "-")
# tomek_df = undummify(tomek_df, "-")

In [ ]:
smote_sex_df = undummify( pd.get_dummies(smote_df, prefix_sep= "-"), prefix_sep = "-", col_list = ["sex"]) 

In [ ]:
tomek_sex_df = undummify( pd.get_dummies(tomek_df, prefix_sep= "-"), prefix_sep = "-", col_list = ["sex"]) 
smote_tomek_sex_df = undummify( pd.get_dummies(smote_tomek_df, prefix_sep= "-"), prefix_sep = "-", col_list = ["sex"]) 

In [ ]:
data_rebalance_smote =  DataRebalance(smote_sex_df, 'sex', None, None, None, smote, None)
data_rebalance_tomek = DataRebalance(tomek_sex_df, 'sex', None, None, None, None, None)
data_rebalance_smote_tomek = DataRebalance(smote_tomek_sex_df, 'sex', None, None, None, None, None)
smote_df = data_rebalance_smote.Rebalance()
tomek_df = data_rebalance_tomek.Rebalance()
# smote_tomek_df = data_rebalance_smote_tomek.Rebalance()

In [ ]:
smote_df = undummify()
tomek_df = undummify()
# smote_tomek_df = undummify()

In [ ]:
from databalanceanalysis.databalanceanalysis.feature_measures import FeatureBalanceMeasure
feat_measures = feature_measures.measures(df)

First look at the feature balance measures for the dataset without applying SMOTE or TOMEK

In [ ]:
feat_measures

After applying SMOTE method, these are the feature balance measures

In [ ]:
feature_measures.measures(smote_df)

Applying applying the SMOTE-Tomek Method these are the feature balance measures.

In [ ]:
feature_measures.measures(smote_tomek_df)

After appying the Tomek Method

In [ ]:
feature_measures.measures(tomek_df)

In [ ]:
from databalanceanalysis.databalanceanalysis.distribution_measures import DistributionBalanceMeasure
dist_measures = DistributionBalanceMeasure(cols_of_interest)
dist_measures.measures(df)


In [ ]:
dist_measures.measures(smote_df)

In [ ]:
dist_measures.measures(smote_tomek_df)

In [ ]:
dist_measures.measures(tomek_df)

In [ ]:
from databalanceanalysis.databalanceanalysis.aggregate_measures import AggregateBalanceMeasure
agg_measures = AggregateBalanceMeasure(cols_of_interest)
agg_measures.measures(df)

In [ ]:
agg_measures.measures(smote_df)

In [ ]:
agg_measures.measures(smote_tomek_df)

In [ ]:
agg_measures.measures(tomek_df)